In [1]:
import astropy.io.fits as fits

import numpy as np
import healpy as hp

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
mask = hp.read_map("../data/Planck/mask_ps_gal40.fits") > 0

ps_mask = hp.read_map("../data/Planck/mask_ps.fits") > 0

shear_mask = hp.read_map("../data/shear_maps_KiDS1000/z0.1-1.2/singlet_mask.fits") > 0

In [3]:
with fits.open("../data/Planck/COM_PCCS_100_R2.01.fits") as hdu:
    l, b = hdu[1].data["GLAT"], hdu[1].data["GLON"]
    extended, flux, flux_err = hdu[1].data["EXTENDED"], hdu[1].data["DETFLUX"], hdu[1].data["DETFLUX_ERR"]

pccs2e =np.zeros(l.size, dtype=bool)

In [4]:
with fits.open("../data/Planck/COM_PCCS_100-excluded_R2.01.fits") as hdu:
    l = np.append(l, hdu[1].data["GLAT"])
    b = np.append(b, hdu[1].data["GLON"])

    extended = np.append(extended, hdu[1].data["EXTENDED"])
    flux = np.append(flux, hdu[1].data["DETFLUX"])
    flux_err = np.append(flux_err, hdu[1].data["DETFLUX_ERR"])

    pccs2e = np.append(pccs2e, np.ones(l.size-pccs2e.size, dtype=bool))

In [16]:
pixel_idx = hp.ang2pix(2048, b, l, lonlat=True)

unmasked_source_ps_mask_idx = (ps_mask)[pixel_idx]
unmasked_source_ps_gal_mask_idx = (mask)[pixel_idx]
unmasked_source_ps_gal_shear_mask_idx = (mask & shear_mask)[pixel_idx]

flux_greater_300_idx = flux > 300


In [21]:
m = unmasked_source_ps_gal_mask_idx & flux_greater_300_idx

print("Unmasked sources (point source and 40% galactic mask) with flux > 300 mJy")
print("Flux [mJy]    Flux/σ_flux  Extended     in PCCS2E   in KiDS footprint")
for e, f, f_err, pe, in_kids_footprint in zip(extended[m],
                                              flux[m], flux_err[m],
                                              pccs2e[m],
                                              unmasked_source_ps_gal_shear_mask_idx[m]):
    print(f"{f:.1f}±{f_err:.1f}    {f/f_err:.1f}          {e>0}         {pe}        {in_kids_footprint}")

Unmasked sources (point source and 40% galactic mask) with flux > 300 mJy
Flux [mJy]    Flux/σ_flux  Extended     in PCCS2E   in KiDS footprint
300.4±61.3    4.9          True         False        False
363.8±61.6    5.9          True         False        False
